In [1]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.express as px 
import ipywidgets as widgets
import numpy as np
from scipy import special
import time
import math as ms
import sys

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery
 
client = BigQuery.getBigQueryClient_TDMScenarios()

In [2]:
# include in all scenario groups
lstIncludeInAll = ['BY','TIP']

# dataframe to create subcategories
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit'],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Auto'     ,'0: Total'                 ],
    ['SchoolBus','0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ]]
,columns=('MODE','modeGroup'))
#dfModeGroups

In [3]:
# FILTER
#strSQLWhere = ' WHERE NOT (t.scenarioID = 33)'
strSQLWhere = ''

# merge scenario data onto transit share data

# read transit summary from biq query
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share AS t" + strSQLWhere).to_dataframe()
#display(dfTransitSummary)

dfRouteSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_summary AS t" + strSQLWhere).to_dataframe()
#display(dfRouteSummary)

# read scenarios data biq query
dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios AS t" + strSQLWhere).to_dataframe()
#display(dfScenarios)

# merge two dataframes
dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge two dataframes
dfRouteSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfRouteSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge to mode groupings dataframe
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
#dfTransitSummaryPlotData


In [4]:
# check to see if all scenarios has same record count
dfTransitSummaryPlotData_Check = dfTransitSummaryPlotData
dfTransitSummaryPlotData_Check.groupby(['tdmVersionWithDate','scenarioGroup','scenarioYear']).agg(COUNT=('TRIPS','count'))

COUNT
tdmVersionWithDate                  scenarioGroup scenarioYear       
WF TDM v8.3.2 - 2022-02-04a         BY            2019            280
                                    RTP           2030            280
                                                  2040            280
                                                  2050            280
                                    TIP           2024            280
WF TDM v8.3.2 - 2022-09-21          BY            2019            280
                                                  2023            280
                                    Needs         2032            280
                                                  2042            280
                                                  2050            280
                                    NoBuild       2032            280
                                                  2042            280
                                                  2050            280
                                    RTP           2032            280
                                                  2042            280
                                                  2050            280
                                    TIP           2028            280
WF TDM v8.3.2 - 2023-02-28          BY            2019            280
                                                  2023            280
                                    RTP           2032            280
                                                  2042            280
                                                  2050            280
                                    TIP           2028            280
WF TDM v9.0 - 2022-12-19            BY            2019            280
                                                  2023            280
                                    EF_RTP        2032            280
                                                  2042            280
                                                  2050            280
                                    Needs         2032            280
                                                  2042            280
                                                  2050            280
                                    NoBuild       2032            280
                                                  2042            280
                                                  2050            280
                                    RTP           2032            280
                                                  2042            280
                                                  2050            280
                                    TIP           2028            280
WF TDM v9.0 - 2023-02-23-MCCalib-r0 BY            2019            280
                                                  2023            280
                                    RTP           2032            280
                                                  2042            280
                                                  2050            280
                                    TIP           2028            280
WF TDM v9.0 - 2023-02-23-MCCalib-r2 BY            2019            280
                                                  2023            280
                                    RTP           2032            280
                                                  2042            280
                                                  2050            280
                                    TIP           2028            280

In [5]:
#PLOTTING FUNCTION

import math

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):

    data = []

    for v in tdmVersionsWithDate:
        for g in scenarioGroups:
            for m in modeGroups:

                # only do if data in dataframe since BY data is concatonated later
                if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin([g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

                    # data for plotting from filtered dataframe}
                    plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin(lstIncludeInAll + [g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))]

                    #display(plotdata)

                    plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                    # fill any NaN values with zeros
                    plotdata = plotdata.fillna(0)

                    #display(plotdata)

                    xplot = plotdata['scenarioYear']
                    yplot = plotdata['TRIPS'       ]

                    trace1 = go.Scatter(
                        x=xplot,
                        y=yplot,
                        mode='markers+lines',
                        name= v + ' ' + g + ' ' + m,
                        marker=dict(size=12,
                                line=dict(width=2,
                                            color='DarkSlateGrey'))#,
                        #stackgroup='one',
                        #groupnorm='percent' # sets the normalization for the sum of the stackgroup
        #                line=dict(
        #                    shape='spline'
        #                )
                    )
                    data.append(trace1)


    layout = go.Layout(
        title='Trips by Mode (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips',
            rangemode = 'tozero'#,
            #range=(0,np.null)
        ),
        xaxis=dict(
            title='Year',
            range=(2018,2051)
        ),
        width=1000,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [6]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTdmVersionWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups     = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes              = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp           = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod             = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionsWithDate = widgets.SelectMultiple(options=lstTdmVersionWithDate, value=(lstTdmVersionWithDate), description='TDM Version'   )
selectScenarioGroups      = widgets.SelectMultiple(options=lstScenarioGroups    , value=('RTP',               ), description='Scenario Group')
selectModes               = widgets.SelectMultiple(options=lstModes             , value=('4: Transit',        ), description='Mode Group'    )
selectTripPurps           = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp          ), description='Purpose'       )
selectPeriods             = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod            ), description='Periods'       )

#custom extents
#xmin = widgets.Text(value='0'  , description='X Min')
#xmax = widgets.Text(value='160' , description='X Max')
#ymin = widgets.Text(value='0'  , description='Y Min')
#ymax = widgets.Text(value='.10', description='Y Max')

widgets.interactive(update_plot, tdmVersionsWithDate=selectTdmVersionsWithDate, scenarioGroups=selectScenarioGroups, modeGroups=selectModes, trippurps=selectTripPurps , periods=selectPeriods)

#ADD RAW OBSERVED CURVES

interactive(children=(SelectMultiple(description='TDM Version', index=(0, 1, 2, 3, 4, 5), options=('WF TDM v8.…

In [31]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea(tdmVersionWithDate, scenarioGroup, modeGroups, trippurps, periods):
    
    #output.clear_output()

    data = []

    modeGroups = sorted(modeGroups)

    for m in modeGroups: 
        # only do if data in dataframe since BY data is concatonated later
        if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) &
                                    (dfTransitSummaryPlotData['scenarioGroup'     ]==scenarioGroup     ) &
                                    (dfTransitSummaryPlotData['modeGroup'         ]==m                 ) &
                                    (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)    ) &
                                    (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)      )].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                (dfTransitSummaryPlotData['modeGroup'         ]==m                                     ) &
                                                (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)                        ) &
                                                (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)                          )]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['TRIPS'       ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= m,
                stackgroup='one'#,
                #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        title=tdmVersionWithDate + ' Trips Mode Split (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=1000,
        height=400
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    #py.offline.iplot(fig2)
    display(fig2)


In [50]:
# MAKE INTERACTIVE CHART
#py.offline.init_notebook_mode(connected=True)

from ipywidgets import Layout

lstTdmVersionsWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes               = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp            = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod              = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]          ), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                              ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))
selectModes              = widgets.SelectMultiple(options=lstModes             , value=(lstModes[7:13]                     ), description='Mode Group'    , layout=Layout(display="flex", flex_flow='row'))
selectTripPurps          = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp                        ), description='Purpose'       , layout=Layout(display="flex", flex_flow='row'))
selectPeriods            = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod                          ), description='Periods'       , layout=Layout(display="flex", flex_flow='row'))


# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup, selectModes, selectTripPurps, selectPeriods])
# create interactive widget
interactive_output = widgets.interactive_output(update_plot_stackedarea, {'tdmVersionWithDate': selectTdmVersionWithDate, 'scenarioGroup':selectScenarioGroup, 'modeGroups':selectModes, 'trippurps':selectTripPurps, 'periods':selectPeriods})

display(hbox, interactive_output)
display(output)
#ADD RAW OBSERVED CURVES

Output()

Output()

# Boardings

In [ ]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea_boardings(tdmVersionWithDate, scenarioGroup):

    data = []

    modes = sorted(dfRouteSummaryWithScenarioData['MODE'].unique())

    for m in modes: 
        # only do if data in dataframe since BY data is concatonated later
        if dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate) &
                                          (dfRouteSummaryWithScenarioData['scenarioGroup'     ]==scenarioGroup     ) &
                                          (dfRouteSummaryWithScenarioData['MODE'              ]==m                 )].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                      (dfRouteSummaryWithScenarioData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                      (dfRouteSummaryWithScenarioData['MODE'              ]==m                                     )]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(BOARDINGS=('DY_BRDA','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['BOARDINGS'   ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= 'Mode ' + str(m),
                stackgroup='one'#,
                #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        title=tdmVersionWithDate + ' Boardings by Mode',
        yaxis=dict(
            title='Trips'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=1000,
        height=400
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig2)


In [ ]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

from ipywidgets import Layout

lstTdmVersionsWithDate = sorted(dfRouteSummaryWithScenarioData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfRouteSummaryWithScenarioData['scenarioGroup'     ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]          ), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                              ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))

widgets.interactive(update_plot_stackedarea_boardings, tdmVersionWithDate=selectTdmVersionWithDate, scenarioGroup=selectScenarioGroup)

#ADD RAW OBSERVED CURVES

interactive(children=(Select(description='TDM Version', index=2, layout=Layout(display='flex', flex_flow='row'…

In [ ]:
dfRouteSummaryWithScenarioData

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear,MODE,NAMEID,NAME,OPERATOR,...,DY_D8_BRDA,DY_W8_BRDA,DY_8_XITB,DY_D9_BRDA,DY_W9_BRDA,DY_9_XITB,DY_D_BRDA,DY_W_BRDA,DY_BRDA,DY_XITB
0,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,5,88.0,S1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.71,43.16,43.87,43.87
1,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,4,47.0,S4,0,...,0.00,0.00,0.00,0.00,0.00,0.00,36.12,1307.91,1344.03,1344.03
2,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,5,88.1,S1-,0,...,0.00,0.00,0.00,0.03,2.52,2.55,1.95,1437.13,1439.08,1439.08
3,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,4,38.0,S17,0,...,0.00,0.06,0.06,0.00,3.73,3.73,2.02,490.75,492.77,492.77
4,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,4,43.0,S31,0,...,0.00,0.00,0.00,0.00,0.00,0.00,7.54,163.55,171.09,171.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7993,6.18,v832,WF TDM v8.3.2 - 2023-02-28,TIP_2028,TIP,2028,9,91.1,UVX-,0,...,1934.24,1312.21,3246.45,360.72,5532.55,5893.27,2294.96,6844.76,9139.72,9139.72
7994,6.18,v832,WF TDM v8.3.2 - 2023-02-28,TIP_2028,TIP,2028,9,91.0,UVX,0,...,628.25,436.12,1064.37,728.80,6251.86,6980.66,1357.05,6687.98,8045.03,8045.03
7995,6.18,v832,WF TDM v8.3.2 - 2023-02-28,TIP_2028,TIP,2028,9,90.0,O603,0,...,354.64,133.12,487.76,194.51,599.73,794.24,549.15,732.85,1282.00,1282.00
7996,6.18,v832,WF TDM v8.3.2 - 2023-02-28,TIP_2028,TIP,2028,9,90.1,O603-,0,...,0.00,267.36,267.36,4.35,594.26,598.61,4.35,861.64,865.99,865.99
